# A simple example

In [1]:
class Person:
    def __init__(self, fname, lname):
        self.firstname = fname
        self.lastname = lname
    
    def printargs(self):
        print(vars(self))
        
    def printname(self):
        print(self. firstname, self.lastname)
        
class Student(Person):
    def __init__(self, fname, lname, year):
        Person.__init__(self, fname, lname)
        self.graduationyear = year
        
class HighschoolID(Student):
    def getid(self):
        identification = 1545321
        self.identification = identification
    def printid(self):
        print(self.identification)
    

In [2]:
# default_solver_parameters = {
#     'degree': defaultdict(lambda: 1),
#     'space' : defaultdict(lambda: 'FunctionSpace')
# }


# class test_dict:
#     def __init__(self, parameters):
#         self.prm = parameters
#         self.setup()
    
#     def setup(self):
#         self.define_dict(self.prm['space'])
    
#     def define_dict(self, space):
#         V = self.V = dict({'i', space['i']})
            
# diction = test_dict(default_solver_parameters)
dictionary = {'x': 2, 'y' : 3}
dictionary2 = {'key' : dictionary}
dictionary2['key']
list1 = {'first', 'second', 'third'}
for i in list1:
    x = dict([('second' , dictionary2)])
print(dictionary2)
print(x)

{'key': {'x': 2, 'y': 3}}
{'second': {'key': {'x': 2, 'y': 3}}}


# PDE Examples

In [3]:
from collections import defaultdict
import copy

In [4]:
default_solver_parameters = {
    'degree': defaultdict(lambda: 1),
    'family': defaultdict(lambda: 'CG'),
    'space' : defaultdict(lambda: 'FunctionSpace'),
    'constrained_domain': None,
    'constriction': defaultdict(lambda: dict(constrained_domain=None)),
    'pdesubsystem': defaultdict(lambda: 1),
    'iteration_type': 'Picard',   # or 'Newton'
    'linear_solver': defaultdict(lambda: 'lu'),
    'precond': defaultdict(lambda: 'default'),
    'omega': defaultdict(lambda: 1.0), # Underrelaxation
    'monitor_convergence': defaultdict(lambda: False), # Monitor Krylov solver
    'time_integration': 'Steady', # or 'Transient'
    'max_iter': 1,
    'max_err': 1e-7,
    'T': 1., # End time for simulation,
    'dt': 0.001,  # timestep,
    'familyname': 'default'    # Name of PDESystem (e.g., Navier-Stokes)
}

def recursive_update(dst, src):
    """Update dict dst with items from src deeply ("deep update")."""
    for key, val in src.items():
        if key in dst and isinstance(val, dict) and isinstance(dst[key], dict):
            dst[key] = recursive_update(dst[key], val)
        else:
            dst[key] = val
    return dst

In [5]:
solver_parameters  = copy.deepcopy(default_solver_parameters)

solver_parameters = recursive_update(solver_parameters, {
        'space': {'u': 'VectorFunctionSpace'},
        'degree': {'u': 2},
        'time_integration': 'Steady',
        'linear_solver': {'up': 'minres'},
        'precond': {'up': 'amg'},
        'monitor_convergence':{'up': True},        
    })
print(default_solver_parameters)
print('\n')
print(solver_parameters)

{'degree': defaultdict(<function <lambda> at 0x7f10d855d2f0>, {}), 'family': defaultdict(<function <lambda> at 0x7f10d855d0d0>, {}), 'space': defaultdict(<function <lambda> at 0x7f10d855d048>, {}), 'constrained_domain': None, 'constriction': defaultdict(<function <lambda> at 0x7f10d855d378>, {}), 'pdesubsystem': defaultdict(<function <lambda> at 0x7f10d855d400>, {}), 'iteration_type': 'Picard', 'linear_solver': defaultdict(<function <lambda> at 0x7f10d855d488>, {}), 'precond': defaultdict(<function <lambda> at 0x7f10d855d510>, {}), 'omega': defaultdict(<function <lambda> at 0x7f10d855d598>, {}), 'monitor_convergence': defaultdict(<function <lambda> at 0x7f10d855d620>, {}), 'time_integration': 'Steady', 'max_iter': 1, 'max_err': 1e-07, 'T': 1.0, 'dt': 0.001, 'familyname': 'default'}


{'degree': defaultdict(<function <lambda> at 0x7f10d855d2f0>, {'u': 2}), 'family': defaultdict(<function <lambda> at 0x7f10d855d0d0>, {}), 'space': defaultdict(<function <lambda> at 0x7f10d855d048>, {'u': 

In [6]:
class PDESystem:
    """
    A base class for solving a system of equations
    """
    def __init__(self, system_composition, parameters):      # removed problem
        self.system_composition = system_composition         # Total system comp.
        self.system_names = []                               # Compounds solved for
        self.names = []                                      # All components
        self.prm = parameters
    
#     #if isinstance(problem, dolfin.cpp.mesh.Mesh):
#         if isinstance(problem, Mesh):    
#             self.problem = None
#             self.mesh = problem
#         else:
#             self.problem = problem
#         self.mesh = problem.mesh
#         self.prm['dt'] = problem.prm['dt']
#         self.prm['time_integration'] = problem.prm['time_integration']
#         self.dt = Constant(self.prm['dt'])
         
        # The following parameters are used by the solve functionality 
        # in case there's no problem instance used to perform the solve.
        self.t = 0                                           # Simulation time
        self.tstep = 0                                       # Time step
        self.total_number_iters = 0                          #
        self.num_timesteps = 0                               #
                
        for sub_system in system_composition: 
            system_name = ''.join(sub_system) 
            self.system_names.append(system_name)
            for n in sub_system:       # Run over all individual components
                self.names.append(n)
    
    def define(self):
        print(vars(self))
                
class Stokes_solver(PDESystem):
     def __init__(self, parameters):
        PDESystem.__init__(self, [['u', 'p']], parameters)
        

In [7]:
solver_parameters  = copy.deepcopy(default_solver_parameters)

solver_parameters = recursive_update(solver_parameters, {
        'space': {'u': 'VectorFunctionSpace'},
        'degree': {'u': 2},
        'time_integration': 'Steady',
        'linear_solver': {'up': 'minres'},
        'precond': {'up': 'amg'},
        'monitor_convergence':{'up': True},        
    })
solver = Stokes_solver(solver_parameters)
solver.define()
print(solver.names, '\n')
print(solver.system_composition, '\n')
print(solver.system_names, '\n')
print(solver.prm, '\n')

{'system_composition': [['u', 'p']], 'system_names': ['up'], 'names': ['u', 'p'], 'prm': {'degree': defaultdict(<function <lambda> at 0x7f10d855d2f0>, {'u': 2}), 'family': defaultdict(<function <lambda> at 0x7f10d855d0d0>, {}), 'space': defaultdict(<function <lambda> at 0x7f10d855d048>, {'u': 'VectorFunctionSpace'}), 'constrained_domain': None, 'constriction': defaultdict(<function <lambda> at 0x7f10d855d378>, {}), 'pdesubsystem': defaultdict(<function <lambda> at 0x7f10d855d400>, {}), 'iteration_type': 'Picard', 'linear_solver': defaultdict(<function <lambda> at 0x7f10d855d488>, {'up': 'minres'}), 'precond': defaultdict(<function <lambda> at 0x7f10d855d510>, {'up': 'amg'}), 'omega': defaultdict(<function <lambda> at 0x7f10d855d598>, {}), 'monitor_convergence': defaultdict(<function <lambda> at 0x7f10d855d620>, {'up': True}), 'time_integration': 'Steady', 'max_iter': 1, 'max_err': 1e-07, 'T': 1.0, 'dt': 0.001, 'familyname': 'default'}, 't': 0, 'tstep': 0, 'total_number_iters': 0, 'num_

In [8]:
solver_parameters  = copy.deepcopy(default_solver_parameters)
solver_parameters = recursive_update(solver_parameters, 
    dict(degree=dict(u=1),
         pdesubsystem=dict(u=1, p=1, velocity_update=1, up=1), 
         linear_solver=dict(u='lu', p='lu', velocity_update='lu'), 
         precond=dict(u='jacobi', p='amg', velocity_update='jacobi'))
         )
solver = Stokes_solver(solver_parameters)
print(solver.names, '\n')
print(solver.system_composition, '\n')
print(solver.system_names, '\n')
print(solver.prm, '\n')
# print(solver.pdesubsystem['up'])

['u', 'p'] 

[['u', 'p']] 

['up'] 

{'degree': defaultdict(<function <lambda> at 0x7f10d855d2f0>, {'u': 1}), 'family': defaultdict(<function <lambda> at 0x7f10d855d0d0>, {}), 'space': defaultdict(<function <lambda> at 0x7f10d855d048>, {}), 'constrained_domain': None, 'constriction': defaultdict(<function <lambda> at 0x7f10d855d378>, {}), 'pdesubsystem': defaultdict(<function <lambda> at 0x7f10d855d400>, {'u': 1, 'p': 1, 'velocity_update': 1, 'up': 1}), 'iteration_type': 'Picard', 'linear_solver': defaultdict(<function <lambda> at 0x7f10d855d488>, {'u': 'lu', 'p': 'lu', 'velocity_update': 'lu'}), 'precond': defaultdict(<function <lambda> at 0x7f10d855d510>, {'u': 'jacobi', 'p': 'amg', 'velocity_update': 'jacobi'}), 'omega': defaultdict(<function <lambda> at 0x7f10d855d598>, {}), 'monitor_convergence': defaultdict(<function <lambda> at 0x7f10d855d620>, {}), 'time_integration': 'Steady', 'max_iter': 1, 'max_err': 1e-07, 'T': 1.0, 'dt': 0.001, 'familyname': 'default'} 



PDESystem Functions:
-setup
    -setup_sub_systems()
    -define_function_spaces
    -add_function_on_timestep()

In [9]:
class PDESystem:
    """
    A base class for solving a system of equations
    """
    def __init__(self, system_composition, parameters):      # removed problem(mesh)
        self.system_composition = system_composition         # Total system comp.
        self.system_names = []                               # Compounds solved for
        self.names = []                                      # All components
        self.prm = parameters
    
        self.t = 0                                           # Simulation time
        self.tstep = 0                                       # Time step
        self.total_number_iters = 0                          #
        self.num_timesteps = 0                               #
                
        for sub_system in system_composition: 
            system_name = ''.join(sub_system) 
            self.system_names.append(system_name)
            for n in sub_system:       # Run over all individual components
                self.names.append(n)
        
        self.setup()
        
    def define(self):
        print(vars(self))
        
    def setup(self):
        self.define_function_spaces(self.prm['degree'], # removed self.mesh
                              self.prm['space'], self.prm['family']) # removed cons
        self.setup_subsystems()
#         if self.prm['time_integration'] == 'Steady':
#             self.add_function_on_timestep()
#         else:
#             self.add_function_on_timestep(['', '1', '2'])
        self.pdesubsystems = dict((name, None) for name in self.system_names)
        self.normalize     = dict((name, None) for name in self.system_names)
        self.bc            = dict((name,   []) for name in self.system_names)
#         if hasattr(self.problem, 'add_pdesystem'):
#             self.problem.add_pdesystem(self, self.prm['familyname'])

    def define_function_spaces(self, degree, space, family):
        """Define functionspaces for names and system_names"""
        V = self.V = dict((name, space[name]#, family[name], degree[name] 
                           ) for name in self.names + ['dq'])

        #Add function space for compound functions for the sub systems
        V.update(dict(
            (sys_name, 'MixedFunctionSpace')
            for sub_sys, sys_name in zip(self.system_composition,
                                         self.system_names) 
                                         if len(sub_sys) > 1))
    def setup_subsystems(self):
        V, sys_names, sys_comp = \
               self.V, self.system_names, self.system_composition
               
        # Create compound functions for the various sub systems
        q = dict((name, 'Trial') for name in sys_names)
        v = dict((name, 'Test')  for name in sys_names)
        
        # Split the various compound functions into sub functions
        for sub_sys, sys_name in zip(sys_comp, sys_names):
            print('sub_sys', sub_sys, ' length', len(sub_sys), '\n')
            print('sys_name', sys_name, '\n')
            if len(sub_sys) > 1:
                q.update(dict((name, 'Trial'+name)
                              for name in sub_sys
                             ))
                v.update(dict((name, 'Test'+name)
                              for name in sub_sys
                              ))

        self.qt, self.vt = q, v
        # Short forms
        for key, value in v.items(): setattr(self, 'v_' + key, value)
        for key, value in q.items(): setattr(self, key, value) 
            
    def solve(self):
        ''' call on the pdesubsystems to solve for their respective variables'''
        for name in self.system_names:
            print('solving pdesubsystems[',name,']')
            x, y = self.pdesubsystems[name].solve_subsystem() 
            print(x, y)
            
class Subdict(dict):
    """Dictionary that looks for missing keys in the solver_namespace"""
    def __init__(self, solver_namespace, sub_name, **kwargs):
        dict.__init__(self, **kwargs)
        self.solver_namespace = solver_namespace
        self.sub_name = sub_name
    
    def __missing__(self, key):
        try:
            self[key] = self.solver_namespace['prm'][key][self.sub_name]
#             info_green("Adding ['{0}']['{1}'] = {2} to pdesubsystem {3}".format(key, 
#                        self.sub_name, self[key], ''.join(self.sub_name)))
        except:
            self[key] = self.solver_namespace['prm'][key]
#             info_green("Adding ['{0}'] = {1} to pdesubsystem {2}".format(key, 
#                        self[key], ''.join(self.sub_name)))
        return self[key]
    
class PDESubsystembase:
    def __init__(self, solver_namespace, sub_system, bcs=[], normalize=None, **kwargs):        
        self.solver_namespace = solver_namespace       
        self.query_args(sub_system)
        self.prm = Subdict(solver_namespace, self.name,
            reassemble_lhs=True,  # True if lhs depends on any q_, q_1, ..
            reassemble_rhs=True,  # True if rhs depends on any q_, q_1, ..
            reassemble_lhs_inner=True, # True if lhs depends on q_
            reassemble_rhs_inner=True, # True if rhs depends on q_
            reassemble_precond=True,
            assemble_system=False,
            max_inner_iter=1,
            max_inner_err=1e-5,
            wall_value=1e-12,
            cache_arrays=False
            )
        self.prm.update(**kwargs)
        self.bcs = bcs
        self.normalize = normalize
        self.exterior_facet_domains = kwargs.get('exterior_facet_domains', None)
        self.F = kwargs.get('F', None)
        self.a = None
        # Define matrix and vector for lhs and rhs respectively
#         self.A = Matrix() # Matrix() can be used for > 0.9.8
#         self.A.initialized = False
#         self.b = Vector()
#         self.B = Matrix() # Possible user defined preconditioner 
#         # Define matrix and vector for unchanging parts of tensor
#         self.A1 = None
#         self.b1 = None        
        self.inner_iter = 0
        
    def query_args(self, sub_system):
        """Check that the correct parameters have been suplied."""
        if not isinstance(sub_system, list):
            raise TypeError("expected a list for sub_system")

        self.name = ''.join(sub_system)
#         self.V = self.solver_namespace[self.name + '_'].function_space()
#         self.x = self.solver_namespace[self.name + '_'].vector()
#         self.sub_system = sub_system
#         self.work = self.get_work_vector()

    def get_solver(self):
        if self.prm['linear_solver'] == 'lu':
            return 'LUSolver'
    
    def get_form(self, form_args):
        F = self.form(**form_args)
        self.F = F
    
        
class PDESubsystem(PDESubsystembase):
    """Base class for most PDESubSystems"""
    def __init__(self, solver_namespace, sub_system, bcs=[], normalize=None, **kwargs):
        PDESubsystembase.__init__(self, solver_namespace, sub_system, bcs, normalize, **kwargs)

#         if not isinstance(self.bcs, list):
#             raise TypeError("expecting a list of boundary conditions")
                        
        self.linear_solver = self.get_solver()
                                        
        self.define()
                                
    def define(self):
        # try and see if we can get the forms first
        form_args = self.solver_namespace.copy()
#         print(form_args)
        self.get_form(form_args)
    
    def solve_subsystem(self):
        print('entered subsystem[',self.name,']')
        print(self.F)
        print(self.linear_solver)
        return self.F, self.linear_solver
    
class Stokes_solver(PDESystem):
    def __init__(self, parameters):
        PDESystem.__init__(self, [['u', 'p']], parameters)
        
        print('vars(self) ', vars(self), '\n')
        self.define()
        
    def define(self):
        self.pdesubsystems['up'] = Stokes(vars(self), ['u', 'p'])

class Stokes(PDESubsystem):
    def form(self, **kwargs):
        return 'form here!'

        
solver_parameters  = copy.deepcopy(default_solver_parameters)
print(solver_parameters, '\n')

solver_parameters = recursive_update(solver_parameters, 
        {
        'space': {'u': 'VectorFunctionSpace'},
        'degree': {'u': 2},
        'time_integration': 'Steady',
        'linear_solver': {'up': 'lu'},
        'precond': {'up': 'amg'},
        'monitor_convergence':{'up': True},}
#          dict(degree=dict(u=1),
#          pdesubsystem=dict(u=1, p=1, velocity_update=1, up=1), 
#          linear_solver=dict(u='lu', p='lu', velocity_update='lu'), 
#          precond=dict(u='jacobi', p='amg', velocity_update='jacobi'))
        
        )
print(solver_parameters, '\n')

solver = Stokes_solver(solver_parameters)

print(solver_parameters, '\n')

print('names: ', solver.names, '\n')
print('system_comp:', solver.system_composition, '\n')
print('system_names: ', solver.system_names, '\n')
# print(solver.prm, '\n')
print(solver.qt, '\n', solver.vt, '\n', solver.V, '\n', solver.pdesubsystems, '\n')
print(solver.pdesubsystems['up'].F, '\n')
print(solver.pdesubsystems['up'].solver_namespace, '\n')
print(solver.pdesubsystems['up'].prm, '\n')
print(solver.pdesubsystems['up'].linear_solver, '\n')

solver.solve()

{'degree': defaultdict(<function <lambda> at 0x7f10d855d2f0>, {}), 'family': defaultdict(<function <lambda> at 0x7f10d855d0d0>, {}), 'space': defaultdict(<function <lambda> at 0x7f10d855d048>, {}), 'constrained_domain': None, 'constriction': defaultdict(<function <lambda> at 0x7f10d855d378>, {}), 'pdesubsystem': defaultdict(<function <lambda> at 0x7f10d855d400>, {}), 'iteration_type': 'Picard', 'linear_solver': defaultdict(<function <lambda> at 0x7f10d855d488>, {}), 'precond': defaultdict(<function <lambda> at 0x7f10d855d510>, {}), 'omega': defaultdict(<function <lambda> at 0x7f10d855d598>, {}), 'monitor_convergence': defaultdict(<function <lambda> at 0x7f10d855d620>, {}), 'time_integration': 'Steady', 'max_iter': 1, 'max_err': 1e-07, 'T': 1.0, 'dt': 0.001, 'familyname': 'default'} 

{'degree': defaultdict(<function <lambda> at 0x7f10d855d2f0>, {'u': 2}), 'family': defaultdict(<function <lambda> at 0x7f10d855d0d0>, {}), 'space': defaultdict(<function <lambda> at 0x7f10d855d048>, {'u': 

In [10]:
name = 'asdf'
func = dict([(name, '1234'+name)])

In [11]:
print(func)

{'asdf': '1234asdf'}


In [12]:
sub_sys = [['u', 'p']]
sys_name = ['up']
x = zip(sub_sys, sys_name)
print(tuple(x))

((['u', 'p'], 'up'),)
